In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import time
import re

from batch_utils.utils_db_alch2 import connectDB
from batch_utils.utils_sql import create_table, update_table, create_typeStr
from batch_utils.utils_dateSeq import batch_sequence
from batch_utils.common import list2sqlstr, _conv2strCol, rm_backward, chunker
from batch_utils import WS_currVal

In [12]:
%qtconsole

In [30]:
bkfil, rtvStart, seq_DT = batch_sequence('backfill', 'M')

Table = 'WSPITCmpIssFData'
Code = ['6751']
Item = '9802'
range_ = 'M2'
cal_dt_ = seq_DT.rename('marketdate')


S0 = pd.Series({'marketdate': 'datetime'})
primary = 'marketdate'
typeStr = create_typeStr(S0, primary=primary)

In [31]:
conn = connectDB(ODBC_NAME="MSSQL_QAD")
create_table(conn, '#Calendar', typeStr, primary=primary)
update_table(conn, '#Calendar', cal_dt_, typeStr, verbose=False)
# Value_ cannot be 0 and EndDate Null at same time
Sql_S = """
Select convert(varchar(8),mm.marketdate,112) as marketdate, dt.Code,
        dt.Item, convert(varchar(8), dt.StartDate,112) as StartDate,
        convert(varchar(8), dt.EndDate,112) as EndDate, dt.Value_
    from #Calendar mm
    left outer join {} dt with (nolock)
    on mm.marketdate >= dt.StartDate
    and mm.marketdate <= isnull(dt.EndDate, dateadd(d,2,GETDATE())) \
""".format(Table)
if range_ is not None:
    Sql_S += (
        '\n    and mm.marketdate ' + 
        f'<= dateadd({range_[0]}, {range_[1:]}, dt.StartDate)\n')
Sql_S += """\
    where dt.Code in ({})
    and dt.Value_ <> -1e+38
    and not (dt.Value_ = 0  and dt.EndDate is NULL)
    and dt.Item='{}'
""".format(list2sqlstr(Code), Item)
DT = pd.read_sql(Sql_S, conn)
conn.close()

In [32]:
DT

,marketdate,Code,Item,StartDate,EndDate,Value_
0,20000126,6751,9802,20000107,20000203,0.66881
1,20000226,6751,9802,20000204,20000302,0.66570
2,20000326,6751,9802,20000303,20000406,0.70751
3,20000426,6751,9802,20000407,20000504,0.83705
4,20000526,6751,9802,20000519,20000608,0.96204
5,20000626,6751,9802,20000609,20000629,1.18831
6,20000726,6751,9802,20000721,20000803,1.14312
7,20000826,6751,9802,20000804,20000831,1.39779
8,20000926,6751,9802,20000901,20000928,1.49589
9,20001026,6751,9802,20000929,20001102,1.81534
